# REFERENCIAS

In [ ]:
#https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html

# FUNÇÕES EM SCALA PARA NORMALIZAÇÃO DE ARRAYS
#https://sparkbyexamples.com/spark/spark-flatten-nested-array-column-to-single-column/

# CALCULO DE MEMÓRIA DE EXECUTORES
#https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html

# AJUSTE DE PARTIÇÕES
#https://sparkbyexamples.com/spark/spark-repartition-vs-coalesce/

# ESTIMATIVA DE MEMÓRIA USADA PELO SPARK
#https://sparkbyexamples.com/spark/spark-difference-between-cache-and-persist/

# IMPORTS

### GENERAL IMPORTS

In [1]:

import json
import bson as bson
import string as s
import pandas as pd
import os.path,inspect, re 
import random as r
from types import SimpleNamespace
import sys
import pathlib
import time
import os
import zipfile
import concurrent.futures
from pymongo import MongoClient
import databricks.koalas as ks


from pyspark.sql.types import *

from pyspark.sql import types as T 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta 
import pyspark
from pyspark.sql import SQLContext  
from pyspark.sql import *


In [2]:
sc = ""
spark = ""
def localInit():
    sc = pyspark.SparkContext()
    spark = pyspark.sql.SparkSession(sc)
    spark.conf.set('spark.Builder.enableHiveSupport',True)
    spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")
    spark.conf.set('spark.sql.execution.arrow.pyspark.enabled' , False)
    spark.conf.set('spark.sql.caseSensitive', True)
    spark.conf.set("spark.sql.shuffle.partitions", "2")

localInit()

## GET MDB SCHEMA

In [ ]:

def getMdbSchema(sampleSize = 50000, mode="local",col=None):
    DATA_PATH = "local_path" 
    if mode =="local":
        with open(DATA_PATH) as f:
            DATA_SCHEMA = T.StructType.fromJson(json.load(f))

    else:
        DATA = spark.read\
        .format( "com.mongodb.spark.sql.DefaultSource")\
        .option('spark.mongodb.input.sampleSize', sampleSize)\
        .option("database", "database")\
        .option("spark.mongodb.collection.collection", col)\
        .load().schema

        DATA


        with open("schema.json", "w") as f:
            json.dump(DATA.jsonValue(), f)

        with open("schema.json") as f:
            DATA_SCHEMA = T.StructType.fromJson(json.load(f))
    return DATA_SCHEMA

## MDB QUERY

In [ ]:
URI = ""
def mdbBuild(URI=None, DATABASE=None,COLLECTION=None):
    client = MongoClient(URI)
    db = client.get_database(DATABASE)
    query_base = db.get_collection(COLLECTION)
    return query_base


## DIR FUNCTIONS

In [ ]:

        
def countDir(self): 
    dcFiles = dbutils.fs.ls(self.DIR)
    df = spark.createDataFrame(sc.parallelize(dcFiles))
    count = df.count()
    displayHTML("DIRECTORY SET AS: ...  %s" % self.DIR)
    displayHTML("TOTAL OF FILES FOUND AFTER SCANNING : ... %s" % count)   

def graphDir(self):
    dcFiles = dbutils.fs.ls(self.DIR)
    df = spark.createDataFrame(sc.parallelize(dcFiles))
    df = df.sort(df.size)
    count = df.count()
    display(df)

def descDir(self):
    #origin
    dcFiles = dbutils.fs.ls(self.dc)
    df = spark.createDataFrame(sc.parallelize(dcFiles))
    df = df.describe()
    display(df)

def filesDir(self):
    #origin
    dcFiles = dbutils.fs.ls(self.DIR)
    return dcFiles       
    


def getInterByIdSingle(self,id=None):
    col = st.findByMongo(COLLECTION='collection')
    col_filter = [
        {
            '$match': {
                '_id': '%s' % id
            }
        }
                ]
    result = col.aggregate(col_filter)
    result = list(result)
    #UTILIZAR ALGUMA FORMA DE PERSISTÊNCIA EM MEMÓRIA PARA ACUMULAR OS RESULTADOS
    self.LIST.append(result)
    size = len(self.INTER_LIST)
    print(size)



%md ALTERNATIVE PLOT